In [1]:
#Downloading necessary libraries
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_auc_score

In [2]:
#Loading training data from file
Trening = pd.read_csv("sc_project_training_data.csv")

In [3]:
#Loading test data from file
Test = pd.read_csv("sc_project_test_data.csv")

In [4]:
#Swaping data from column GameType to numeric
#New data - integers from 1 to 6 inclusive - saving in new column
df = pd.concat([Trening['GameType'], Test['GameType']])
layer = tf.keras.layers.StringLookup()
layer.adapt(df)
Trening['TypeofGame'] = layer(Trening['GameType'])
Test['TypeofGame'] = layer(Test['GameType'])

In [5]:
#Swaping data from column Winner to numeric
Trening['Wynik'] = 0
Trening.loc[Trening['Winner'] == 'A', 'Wynik'] = 1

In [6]:
#Deleting duplicated data and separating results from parameters in training set
#Receiving 3 sets, all with numeric data
#First one with training parameters
#Second one with information who won for training parameters
#Third one with test parameters
Test = Test.drop(columns = ['GameType'])
Wynik = Trening['Wynik']
Trening = Trening.drop(columns = ['GameID', 'Winner', 'GameType', 'Wynik'])

In [7]:
#Normalizing data
scaler = MinMaxScaler()
Trening_skalowany = scaler.fit_transform(Trening)
Test_skalowany = scaler.fit_transform(Test)

In [8]:
#Defining model
model = Sequential()
model.add(Dense(32, input_shape=(158,), activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#The model is neural network made up of input layer, two hidden layers and output layer
#Size of input layer is 158 - it's the number of parameters, which result depends on
#Both hidden layers are the same - they are made up of 32 neuron each
#The function of activation is relu
#In the end there is input layer with function of activation sigmoid

In [9]:
#Compiling model
#As the function of loss we take binary crossentropy
#Optimizing with stochastic gradient descent
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['AUC'])

In [10]:
#Training model
model.fit(Trening_skalowany, Wynik, epochs=100, batch_size=32)

Epoch 1/100
1074/1074 [==============================] - 3s 2ms/step - loss: 0.6659 - auc: 0.5732
Epoch 2/100
1074/1074 [==============================] - 2s 2ms/step - loss: 0.6514 - auc: 0.6384
Epoch 3/100
1074/1074 [==============================] - 2s 2ms/step - loss: 0.6360 - auc: 0.6543
Epoch 4/100
1074/1074 [==============================] - 3s 2ms/step - loss: 0.6269 - auc: 0.6631
Epoch 5/100
1074/1074 [==============================] - 2s 2ms/step - loss: 0.6235 - auc: 0.6681
Epoch 6/100
1074/1074 [==============================] - 2s 2ms/step - loss: 0.6211 - auc: 0.6713
Epoch 7/100
1074/1074 [==============================] - 2s 2ms/step - loss: 0.6193 - auc: 0.6744
Epoch 8/100
1074/1074 [==============================] - 2s 2ms/step - loss: 0.6181 - auc: 0.6757
Epoch 9/100
1074/1074 [==============================] - 2s 2ms/step - loss: 0.6173 - auc: 0.6769
Epoch 10/100
1074/1074 [==============================] - 2s 2ms/step - loss: 0.6167 - auc: 0.6779
Epoch 11/100
1074/1

In [11]:
#Putting test set through the model
predictions = model.predict(Test_skalowany)

336/336 [==============================] - 1s 993us/step


In [12]:
#Saving results to text file
f = open("projekt_sgd_1.txt", "w")
for row in predictions:
    np.savetxt(f, row, fmt='%1.10f')
f.close()

In [13]:
#Separating set to training set and validation set
#Prediction
dane_train, dane_test, wynik_train, wynik_test = train_test_split(Trening_skalowany, Wynik)
pred = model.predict(dane_test)
roc_auc_score(wynik_test, pred)

269/269 [==============================] - 0s 1ms/step


0.7050425901891214

In [13]:
#On 10% of test set we get conformity 0.6978 on average